
# Conversió de fitxers SMILES dels repos de CoCoGraph i MolForge a format d'entrada pels nostres notebooks

Aquest notebook converteix fitxers **Raw MolForge** i **Raw CoCoGraph** al format
`id,SMILES_input` que fa servir MolForge com a entrada.


In [10]:
# Canvi al directori base del projecte
import os
os.chdir("/export/home/ddiestre/MolForge_Testing")

In [11]:
# Imports bàsics i definició de rutes
import pandas as pd

# Rutes d'entrada
input_path_MF = "data/SMILES/raw/MolForge/ECFP4.smiles.test"
input_path_CCG = "data/SMILES/raw/CoCoGraph/novel_molecules.txt"

# Rutes de sortida (fitxers CSV amb columnes id,SMILES_input)
output_path_MF = "data/SMILES/MolForge_2.csv"
output_path_CCG = "data/SMILES/CoCoGraph_2.csv"


## Raw MolForge

- Fitxer d'entrada: `ECFP4.smiles.test` (dues columnes sense títol).  
- Només es conserva **la primera columna**, que es renomena a `SMILES_input`.  
- Es genera un CSV amb columnes `id,SMILES_input`, on `id` comença a 1.


In [12]:
# Llegim el fitxer amb dues columnes sense capçalera
# (separador: tabulador entre SMILES i fingerprint)
df_mf_raw = pd.read_csv(
    input_path_MF,
    sep="\t",
    header=None,
    engine="python"
)

# Nombre total de files del fitxer original
num_files_mf = len(df_mf_raw)
print(f"Nombre total de files al fitxer Raw MolForge: {num_files_mf}")

# Mostrem algunes files per inspeccionar
df_mf_raw.head()

Nombre total de files al fitxer Raw MolForge: 10000


,0,1
0,C C O C 1 = C ( C = C ( C = C 1 ) C ( C ( C ) ...,1 80 94 114 237 241 255 294 392 411 425 695 74...
1,C 1 = C C = C ( C = C 1 ) C 2 C = C ( N C 3 = ...,97 101 314 378 389 442 501 650 728 817 896 909...
2,C O C 1 = C C = C ( C = C 1 ) C ( = O ) C 2 = ...,9 45 78 89 145 203 322 548 586 650 695 718 760...
3,C 1 C N ( C C N 1 C C 2 = C C 3 = C ( C = C 2 ...,74 80 116 197 255 265 310 341 378 407 437 441 ...
4,C C 1 = C C = C ( C = C 1 ) S ( = O ) ( = O ) ...,94 186 210 227 235 249 319 335 348 350 389 440...


In [13]:
# Paràmetres de selecció
n_files_mf = 1000          # nombre de files que vols (posa None per totes)
use_random_mf = False       # True -> agafa files aleatòries, False -> primeres n_files_mf
random_seed = 42           # seed perquè el random sigui reproduïble

In [14]:
# Conversió de Raw MolForge a CSV id,SMILES_input

if n_files_mf is not None:
    if use_random_mf:
        # Mostra aleatòria de n_files_mf files
        df_mf_sel = df_mf_raw.sample(
            n=n_files_mf,
            random_state=random_seed
        ).copy()
    else:
        # Primeres n_files_mf files
        df_mf_sel = df_mf_raw.iloc[:n_files_mf].copy()
else:
    # Totes les files
    df_mf_sel = df_mf_raw.copy()

# Ens quedem només amb la primera columna i la renombrem
df_mf = df_mf_sel.iloc[:, 0].to_frame(name="SMILES_input")

# Eliminem espais sobrants al principi i final (per si de cas)
df_mf["SMILES_input"] = df_mf["SMILES_input"].str.strip()

# Afegim la columna id començant per 1
df_mf.insert(0, "id", range(1, len(df_mf) + 1))

# Definim 'id' com a índex
df_mf.set_index("id", inplace=True)

# Guardem en format CSV
df_mf.to_csv(output_path_MF)

# Mostrem les primeres files per comprovar el format
df_mf.head()


,SMILES_input
id,
1,C C O C 1 = C ( C = C ( C = C 1 ) C ( C ( C ) ...
2,C 1 = C C = C ( C = C 1 ) C 2 C = C ( N C 3 = ...
3,C O C 1 = C C = C ( C = C 1 ) C ( = O ) C 2 = ...
4,C 1 C N ( C C N 1 C C 2 = C C 3 = C ( C = C 2 ...
5,C C 1 = C C = C ( C = C 1 ) S ( = O ) ( = O ) ...



## Raw CoCoGraph

- Fitxer d'entrada: `novel_molecules.txt` (una única columna amb títol `smiles`).  
- La columna `smiles` es renomena a `SMILES_input`.  
- Es genera un CSV amb columnes `id,SMILES_input`, on `id` comença a 1.


In [16]:
# Llegim el fitxer amb una columna anomenada 'smiles'
df_ccg_raw = pd.read_csv(input_path_CCG)

# Nombre total de files del fitxer original
num_files_ccg = len(df_ccg_raw)
print(f"Nombre total de files al fitxer Raw CoCoGraph: {num_files_ccg}")

# Mostrem algunes files per inspeccionar
df_ccg_raw.head()

Nombre total de files al fitxer Raw CoCoGraph: 8059352


,smiles
0,CCCCCN(CCCCc1c2cc3sc2nn13)C(=O)c1ccc(OCC)c(Cl)c1
1,O=C(Nc1n[nH]c2ccc(Br)cc12)N1CCCC1
2,CCN(C(=O)N(C)c1ccccc1OC)C(=Cc1ccc(OC)cc1)c1ccc...
3,CC1(Cl)OC(=O)C1(C)Cl
4,CCc1cc(C=CC(C)=Cc2cc(OC)ccc2C)ccc1C


In [17]:
# Paràmetres de selecció
n_files_ccg = 1000         # nombre de files que vols (posa None per totes)
use_random_ccg = False      # True -> agafa files aleatòries, False -> primeres n_files_ccg
random_seed = 42           # pots reutilitzar el mateix seed o canviar-lo

In [18]:
# Conversió de Raw CoCoGraph a CSV id,SMILES_input

if n_files_ccg is not None:
    if use_random_ccg:
        # Mostra aleatòria de n_files_ccg files
        df_ccg_sel = df_ccg_raw.sample(
            n=n_files_ccg,
            random_state=random_seed
        ).copy()
    else:
        # Primeres n_files_ccg files
        df_ccg_sel = df_ccg_raw.iloc[:n_files_ccg].copy()
else:
    # Totes les files
    df_ccg_sel = df_ccg_raw.copy()

# Renombrem la columna a SMILES_input
if "smiles" in df_ccg_sel.columns:
    df_ccg = df_ccg_sel.rename(columns={"smiles": "SMILES_input"})
else:
    # Si per algun motiu el nom és diferent, agafem la primera columna
    first_col = df_ccg_sel.columns[0]
    df_ccg = df_ccg_sel.rename(columns={first_col: "SMILES_input"})

# Afegim la columna id començant per 1
df_ccg.insert(0, "id", range(1, len(df_ccg) + 1))

# Definim 'id' com a índex
df_ccg.set_index("id", inplace=True)

# Guardem en format CSV (ara l'índex és la columna id)
df_ccg.to_csv(output_path_CCG)

# Mostrem les primeres files per comprovar el format
df_ccg.head()


,SMILES_input
id,
1,CCCCCN(CCCCc1c2cc3sc2nn13)C(=O)c1ccc(OCC)c(Cl)c1
2,O=C(Nc1n[nH]c2ccc(Br)cc12)N1CCCC1
3,CCN(C(=O)N(C)c1ccccc1OC)C(=Cc1ccc(OC)cc1)c1ccc...
4,CC1(Cl)OC(=O)C1(C)Cl
5,CCc1cc(C=CC(C)=Cc2cc(OC)ccc2C)ccc1C
